In [6]:
import sys

sys.path.insert(0, '/Users/elinaisrayelyan/PycharmProjects/capstone_project')
import torch.nn as nn
import torch.optim as optim
from torchmetrics.classification import BinaryAUROC
from modules.model.main import ConvCatNet, model_train_loop, model_test_loop
from modules.data.preprocessing import PreProcess

import torch
import tenseal as ts

from modules.encryption.keys import create_ctx


def size_is_ok():
    img = preprocessor.cuda_image_array.copy()[0][0]

    conv1_weight = net.conv1.weight.data.view(

        -1, net.conv1.kernel_size[0],
        net.conv1.kernel_size[1]
    ).tolist()
    conv1_bias = net.conv1.bias.data.tolist()
    nk = img.tolist().copy()[0]

    x_enc_0, windows_nb_0 = ts.im2col_encoding(
        context, nk, 3,
        3, 1
    )
    y = x_enc_0.conv2d_im2col(conv1_weight[0], windows_nb_0) + conv1_bias[0]
    print(y)
def almost_equal(vec1, vec2, m_pow_ten):
    if len(vec1) != len(vec2):
        return False

    upper_bound = pow(10, -m_pow_ten)
    for v1, v2 in zip(vec1, vec2):
        if abs(v1 - v2) > upper_bound:
            return False
    return True

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 20  # poly degree /2 should be bigger than image to columns for conv size
# context, server_context = create_ctx(bits_scale=33,poly_mod_degree=32768, num_mul=9)  # scale up as working with floats,
context, server_context = create_ctx(poly_mod_degree=32768,num_mul=8)

In [8]:
preprocessor = PreProcess(image_size=image_size, max_num_images=10000)
train_dataloader, test_dataloader = preprocessor.preprocess()

100%|██████████| 1920/1920 [10:22<00:00,  3.08it/s]


In [9]:

net = ConvCatNet(image_size=image_size).to(device)
print(net)

ConvCatNet(
  (lrelu): LeakyReLU(negative_slope=0.1)
  (conv1): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=800, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)
ConvCatNet(
  (lrelu): LeakyReLU(negative_slope=0.1)
  (conv1): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=800, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


In [10]:
size_is_ok()

In [11]:
EPOCHS = 100
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
bauc = BinaryAUROC(thresholds=None)

In [12]:

# MODEL learns the trivial case of all 1s, which means it has capacity to learn:

for epoch in range(2):  # loop over the dataset multiple times

    model_train_loop(net, train_dataloader, optimizer, criterion, bauc, epoch)
    model_test_loop(net, test_dataloader, criterion, bauc, epoch)


# print('Finished Training')

[1,   960] loss: 489.666, ROC: 0.575, F1_healthy: 0.056

[1,   960] Validation loss: 472.891, ROC: 0.716, F1_healthy: 0.000 

[2,   960] loss: 461.275, ROC: 0.647, F1_healthy: 0.000

[2,   960] Validation loss: 459.801, ROC: 0.738, F1_healthy: 0.000 



In [13]:
conv1 = net.conv1.weight.data.view(
            -1, net.conv1.kernel_size[0],
            net.conv1.kernel_size[1]
        ).tolist()
conv1_bias = net.conv1.bias.data.tolist()


In [14]:
img = preprocessor.cuda_image_array[0][0]
img_1 = torch.nn.functional.pad(img, (1, 1, 1, 1), "constant", 0)
img_2 = torch.nn.functional.pad(img, (2, 2, 2, 2), "constant", 0)

kernel_shape = net.conv1.kernel_size

data_0 = img_1[0]
data_1 = img_1[1]
data_2 = img_1[2]

data_2_0 = img_2[0]
data_2_1 = img_2[1]
data_2_2 = img_2[2]
print(data_1.shape)
x_enc_0, windows_nb_0 = ts.im2col_encoding(
    context, data_0, kernel_shape[0],
    kernel_shape[1], 1
)
x_enc_1, windows_nb_1 = ts.im2col_encoding(
    context, data_1, kernel_shape[0],
    kernel_shape[1], 1
)
x_enc_2, windows_nb_2 = ts.im2col_encoding(
    context, data_2, kernel_shape[0],
    kernel_shape[1], 1
)

x_enc_1_0, windows_nb_1_0 = ts.im2col_encoding(
    context, data_2_0, 5,
    5, 1
)
x_enc_1_1, windows_nb_1_1 = ts.im2col_encoding(
    context, data_2_1, 5,
    5, 1
)
x_enc_1_2, windows_nb_1_2 = ts.im2col_encoding(
    context, data_2_2, 5,
    5, 1
)
enc_x = (x_enc_0, x_enc_1, x_enc_2, x_enc_1_0, x_enc_1_1, x_enc_1_2)


torch.Size([22, 22])


In [15]:
x_enc_1.shape

[6400]

In [16]:
enc_channels = 0
for ind, kernel in enumerate(conv1):
    y = enc_x[ind].conv2d_im2col(kernel, windows_nb_1)
    print("one channel")
    print(y.shape)
    enc_channels += y
    # enc_channels.append(y)

one channel
[400]
one channel
[400]
one channel
[400]


In [17]:
net.conv1(img).flatten()

tensor([-5.6807e-02, -5.6807e-02, -6.2843e-02, -6.5623e-02, -5.7546e-02,
        -5.7790e-02, -1.6659e-01, -2.5424e-01, -1.9792e-01, -3.4995e-01,
         1.6504e-01,  2.5949e-01,  9.6217e-02,  2.1638e-01, -5.2511e-02,
        -5.6358e-02, -5.2665e-02, -5.0746e-02, -5.6807e-02, -5.6807e-02,
        -5.6807e-02, -5.6807e-02, -6.0008e-02, -6.1620e-02, -5.4390e-02,
        -1.5453e-01, -4.5135e-01, -3.1491e-01,  1.1845e-01,  6.9145e-02,
         1.0343e-01,  3.0798e-02,  9.1081e-02,  1.9284e-01,  1.0346e-01,
         3.1712e-01, -5.1388e-02, -5.0208e-02, -5.6807e-02, -5.6807e-02,
        -5.6807e-02, -5.6807e-02, -6.0587e-02, -6.1687e-02, -1.5257e-01,
        -4.3462e-01, -2.6354e-01,  7.0232e-02,  1.9292e-01,  7.1254e-02,
         1.0753e-01,  8.1855e-02, -3.2597e-02, -6.3225e-02,  3.7286e-01,
         6.7991e-01, -5.0474e-02, -4.9374e-02, -5.6807e-02, -5.6807e-02,
        -5.6807e-02, -5.6807e-02, -6.0587e-02, -6.2747e-02, -3.3974e-01,
        -4.5832e-01,  4.0309e-02,  1.1285e-01,  1.1

In [18]:
(enc_channels +  conv1_bias[0]).decrypt()


[-0.05682241468878611,
 -0.056811950169762176,
 -0.06284025609843516,
 -0.06562400700252999,
 -0.05754558153616701,
 -0.05779014848341631,
 -0.16659474015059839,
 -0.2542370588964602,
 -0.19791828166882447,
 -0.34995185669739126,
 0.16504282298648687,
 0.25948817841465555,
 0.09621749754950946,
 0.21637669738049384,
 -0.052510999303867184,
 -0.05635794124259542,
 -0.052665186688317336,
 -0.05074570831777708,
 -0.056806713186367916,
 -0.056806304963506755,
 -0.05680665005686409,
 -0.05680675094042005,
 -0.06000780351788428,
 -0.061619715581996776,
 -0.05439054992568382,
 -0.15452638326401066,
 -0.45135331945977863,
 -0.3149085477908481,
 0.11845459851497527,
 0.06914534331287442,
 0.10342805741057229,
 0.030799493182893947,
 0.09108127371743824,
 0.192837926522796,
 0.103460830594685,
 0.3171179847219999,
 -0.05138783349380546,
 -0.0502084636188372,
 -0.056806881951142724,
 -0.05680643925521661,
 -0.05680633392678096,
 -0.05680587695859234,
 -0.06058691507703496,
 -0.06168656994385763,


In [19]:
(enc_channels +  conv1_bias[0]).decrypt()

[-0.05682241468878611,
 -0.056811950169762176,
 -0.06284025609843516,
 -0.06562400700252999,
 -0.05754558153616701,
 -0.05779014848341631,
 -0.16659474015059839,
 -0.2542370588964602,
 -0.19791828166882447,
 -0.34995185669739126,
 0.16504282298648687,
 0.25948817841465555,
 0.09621749754950946,
 0.21637669738049384,
 -0.052510999303867184,
 -0.05635794124259542,
 -0.052665186688317336,
 -0.05074570831777708,
 -0.056806713186367916,
 -0.056806304963506755,
 -0.05680665005686409,
 -0.05680675094042005,
 -0.06000780351788428,
 -0.061619715581996776,
 -0.05439054992568382,
 -0.15452638326401066,
 -0.45135331945977863,
 -0.3149085477908481,
 0.11845459851497527,
 0.06914534331287442,
 0.10342805741057229,
 0.030799493182893947,
 0.09108127371743824,
 0.192837926522796,
 0.103460830594685,
 0.3171179847219999,
 -0.05138783349380546,
 -0.0502084636188372,
 -0.056806881951142724,
 -0.05680643925521661,
 -0.05680633392678096,
 -0.05680587695859234,
 -0.06058691507703496,
 -0.06168656994385763,


# enc

In [20]:
class EncConvNet:

    def __init__(self, torch_nn):
        self.conv1_weight = torch_nn.conv1.weight.data.view(
            -1, torch_nn.conv1.kernel_size[0],
            torch_nn.conv1.kernel_size[1]
        ).tolist()
        self.conv1_bias = torch_nn.conv1.bias.data.tolist()

        self.conv2_weight = torch_nn.conv2.weight.data.view(
            -1, torch_nn.conv2.kernel_size[0],
            torch_nn.conv2.kernel_size[1]
        ).tolist()
        self.conv2_bias = torch_nn.conv2.bias.data.tolist()

        self.fc1_weight = torch_nn.fc1.weight.T.data
        self.fc1_bias = torch_nn.fc1.bias.data.tolist()

        self.fc2_weight = torch_nn.fc2.weight.T.data.tolist()
        self.fc2_bias = torch_nn.fc2.bias.data.tolist()

    def forward(self, enc_x, windows_nb, windows_nb_2=0):
        # conv layer
        enc_channels = 0
        for ind, kernel in enumerate(self.conv1_weight):
            y = enc_x[ind].conv2d_im2col(kernel, windows_nb)

            print("one channel")
            print(y.shape)
            enc_channels += y
            # enc_channels.append(y)

        print("the conv shape")
        print(enc_channels.shape)
        enc_channels +=  self.conv1_bias[0]
        enc_channels_2 = 0
        for ind, kernel in enumerate(self.conv2_weight):
            y = enc_x[ind + 3].conv2d_im2col(kernel, windows_nb_2)
            print("one channel")
            print(y.shape)
            enc_channels_2 += y
        enc_channels_2 +=  self.conv2_bias[0]

        enc_x_conv_2 = ts.CKKSVector.pack_vectors([enc_channels, enc_channels_2])
        print(enc_x_conv_2.shape)

        # pack all channels into a single flattened vector
        # enc_x = ts.CKKSVector.pack_vectors(enc_channels)
        enc_x = enc_x_conv_2
        print((self.fc1_weight).shape)
        # fc1 layer
        enc_x = enc_x.mm(self.fc1_weight) + self.fc1_bias
        print(enc_x.shape)
        # square activation
        enc_x = EncConvNet.relu(enc_x)
        # fc2 layer
        enc_x = enc_x.mm(self.fc2_weight) + self.fc2_bias
        enc_x = EncConvNet.sigmoid(enc_x)
        return enc_x

    @staticmethod
    def sigmoid(enc_x):
        # We use the polynomial approximation of degree 3
        # sigmoid(x) = 0.5 + 0.197 * x - 0.004 * x^3
        # from https://eprint.iacr.org/2018/462.pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.5, 0.197, 0, -0.004])

    @staticmethod
    def relu(enc_x):
        # We use the polynomial approximation of degree 3
        # relu(x) = 0.47+0.5*x+0.09*(x**2)-1.7*e-10*(x**3)
        # from https://openreview.net/attachment?id=rkxsgkHKvH&name=original_pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.47, 0.5, 0.09, -0.0000000007])

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [21]:
enc_net = EncConvNet(net)

In [22]:
real_outputs = []
for idx, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    outputs = net(inputs)
    real_outputs.append(outputs)

In [23]:
img = preprocessor.cuda_image_array[0][0]


In [24]:
real_out = net(img)

In [25]:
len(real_outputs)

960

In [ ]:
enc_outputs = []
for data in test_dataloader:
    img, labels = data
    print(img.shape)
    img = img[0]
    # img = preprocessor.cuda_image_array[i][0]
    img_1 = torch.nn.functional.pad(img, (1, 1, 1, 1), "constant", 0)
    img_2 = torch.nn.functional.pad(img, (2, 2, 2, 2), "constant", 0)

    kernel_shape = net.conv1.kernel_size

    data_0 = img_1[0]
    data_1 = img_1[1]
    data_2 = img_1[2]

    data_2_0 = img_2[0]
    data_2_1 = img_2[1]
    data_2_2 = img_2[2]
    print(data_1.shape)
    x_enc_0, windows_nb_0 = ts.im2col_encoding(
        context, data_0, kernel_shape[0],
        kernel_shape[1], 1
    )
    x_enc_1, windows_nb_1 = ts.im2col_encoding(
        context, data_1, kernel_shape[0],
        kernel_shape[1], 1
    )
    x_enc_2, windows_nb_2 = ts.im2col_encoding(
        context, data_2, kernel_shape[0],
        kernel_shape[1], 1
    )

    x_enc_1_0, windows_nb_1_0 = ts.im2col_encoding(
        context, data_2_0, 5,
        5, 1
    )
    x_enc_1_1, windows_nb_1_1 = ts.im2col_encoding(
        context, data_2_1, 5,
        5, 1
    )
    x_enc_1_2, windows_nb_1_2 = ts.im2col_encoding(
        context, data_2_2, 5,
        5, 1
    )
    x_enc = (x_enc_0, x_enc_1, x_enc_2, x_enc_1_0, x_enc_1_1, x_enc_1_2)
    enc_output = enc_net(x_enc, windows_nb_1, windows_nb_1_2)
    enc_outputs.append(enc_output.decrypt())

torch.Size([1, 3, 20, 20])
torch.Size([22, 22])
one channel
[400]
one channel
[400]
one channel
[400]
the conv shape
[400]
one channel
[400]
one channel
[400]
one channel
[400]
[800]
torch.Size([800, 128])
[128]
torch.Size([1, 3, 20, 20])
torch.Size([22, 22])
one channel
[400]
one channel
[400]
one channel
[400]
the conv shape
[400]
one channel
[400]
one channel
[400]
one channel
[400]
[800]
torch.Size([800, 128])
[128]
torch.Size([1, 3, 20, 20])
torch.Size([22, 22])
one channel
[400]
one channel
[400]
one channel
[400]
the conv shape
[400]
one channel
[400]
one channel
[400]
one channel
[400]
[800]
torch.Size([800, 128])
[128]
torch.Size([1, 3, 20, 20])
torch.Size([22, 22])
one channel
[400]
one channel
[400]
one channel
[400]
the conv shape
[400]
one channel
[400]
one channel
[400]
one channel
[400]
[800]
torch.Size([800, 128])
[128]
torch.Size([1, 3, 20, 20])
torch.Size([22, 22])
one channel
[400]
one channel
[400]
one channel
[400]
the conv shape
[400]
one channel
[400]
one channel

In [ ]:
import numpy as np

np.average(
    np.abs(np.array(enc_outputs) - np.array([torch.flatten(i)[0].detach().numpy() for i in real_outputs])
           )
)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(
    np.abs(np.array(enc_outputs) - np.array([torch.flatten(i)[0].detach().numpy() for i in real_outputs])
           )
)

In [ ]:
!pip install tabulate
!pip install pytest

import tenseal as ts
import random
import pytest
from IPython.display import HTML, display
import tabulate


def decrypt(enc):
    return enc.decrypt()

In [ ]:
data = [random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [
    ["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# img = preprocessor.cuda_image_array[0][0]
img_1 = torch.nn.functional.pad(img, (1, 1, 1, 1), "constant", 0)
img_2 = torch.nn.functional.pad(img, (2, 2, 2, 2), "constant", 0)

kernel_shape = net.conv1.kernel_size

data_0 = img_1[0]
data_1 = img_1[1]
data_2 = img_1[2]

data_2_0 = img_2[0]
data_2_1 = img_2[1]
data_2_2 = img_2[2]

for (poly_mod, coeff_mod_bit_sizes, prec) in [
    # (8192, [60, 40, 60], 40),

    (16384, [40, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 40], 30),
    (16384, [60, 30, 30, 30, 30, 30, 30, 30, 30, 30, 60], 30),
    # (16384, [40,35,35,35,35,35,35,35,35,35,35,35,35, 40], 35),
    (16384, [40, 30, 30, 30, 30, 30, 30, 30, 30, 30, 40], 26),
    (16384, [60, 26,26,26,26,26,26,26,26,26,26, 60], 26),
    (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40,  40, 40, 60], 40),
    (32768, [60,26, 26, 26, 26, 26, 26, 26, 26, 26,26, 26, 26, 60], 26),

    (16384, [40, 26, 26, 26, 26, 26, 26, 26, 26, 26, 40], 26),
    (16384, [40, 29,29,29,29,29,29,29,29,29,29, 40], 29)
    # (16384, [60, 40,40,40,40,40,40,40, 60], 40),

    # (8192, [40, 21, 21, 21, 21, 21, 21, 40], 40),
    # (8192, [40, 21, 21, 21, 21, 21, 21, 40], 21),
    # (8192, [40, 20, 40], 40),
    # (8192, [20, 20, 20], 38),
    # (8192, [60, 60], 38),
    # (8192, [40, 40], 38),
    # (8192, [17, 17], 15),

]:
    print(poly_mod, coeff_mod_bit_sizes, prec)
    val_str = "[0 - 2^{}]".format(0)

    context = ts.context(
        scheme=ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=poly_mod,
        coeff_mod_bit_sizes=coeff_mod_bit_sizes,
        encryption_type=enc_type,
    )

    context.global_scale = 2 ** prec
    context.generate_galois_keys()

    # try:
    #     ckks_vec = ts.ckks_vector(context, data)
    # except BaseException as e:
    #     ct_size_benchmarks.append(
    #         [val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "encrypt", "encryption failed"])
    #     continue

    x_enc_0, windows_nb_0 = ts.im2col_encoding(
        context, data_0, kernel_shape[0],
        kernel_shape[1], 1
    )
    x_enc_1, windows_nb_1 = ts.im2col_encoding(
        context, data_1, kernel_shape[0],
        kernel_shape[1], 1
    )
    x_enc_2, windows_nb_2 = ts.im2col_encoding(
        context, data_2, kernel_shape[0],
        kernel_shape[1], 1
    )

    x_enc_1_0, windows_nb_1_0 = ts.im2col_encoding(
        context, data_2_0, 5,
        5, 1
    )
    x_enc_1_1, windows_nb_1_1 = ts.im2col_encoding(
        context, data_2_1, 5,
        5, 1
    )
    x_enc_1_2, windows_nb_1_2 = ts.im2col_encoding(
        context, data_2_2, 5,
        5, 1
    )
    x_enc = (x_enc_0, x_enc_1, x_enc_2, x_enc_1_0, x_enc_1_1, x_enc_1_2)
    enc_output = enc_net(x_enc, windows_nb_1, windows_nb_1_2)

    # ckks_sum = ckks_vec + ckks_vec
    decrypted = decrypt(enc_output)
    for dec_prec in reversed(range(prec)):
        if pytest.approx(decrypted, abs=2 ** -dec_prec) ==  np.array([torch.flatten(i)[0].detach().numpy() for i in real_outputs]):
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "enc",
                                       "decryption prec 2 ** {}".format(-dec_prec)])
            break

display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))